In [20]:
from supabase import create_client
import json
import pandas as pd
from datetime import date
import statsmodels
import numpy as np
import datetime
from datetime import datetime
import time
import pandas as pd
import numpy as np
import statistics
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module
import time
from selenium import webdriver
from selenium.webdriver import ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.common.keys import Keys

In [21]:
def driver_code():
    Capabilities = DesiredCapabilities.CHROME
    Capabilities["pageLoadStrategy"] = "normal"
    options = ChromeOptions()

    useragentarray = [
        "Mozilla/5.0 (Linux; Android 13) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.5672.76 Mobile Safari/537.36"
    ]

    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    # options.add_argument(f"--user-data-dir=./profile{driver_num}")

    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option("useAutomationExtension", False)
    options.add_argument("disable-infobars")
    options.add_argument("disable-blink-features=AutomationControlled")

    driver = webdriver.Chrome(
        '/Users/paulconnolly/Desktop/ChromeDriver/chromedriver',
        options=options,
        desired_capabilities=Capabilities,
    )
    driver.execute_script(
        "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"
    )

    driver.execute_cdp_cmd(
        "Network.setUserAgentOverride", {"userAgent": useragentarray[0]}
    )

    options.add_argument("--disable-popup-blocking")
    #     driver.execute_script(
    #         """setTimeout(() => window.location.href="https://www.bet365.com.au", 100)"""
    #     )
    driver.get("https://www.bet365.com/#/AC/B1/C1/D1002/E91422157/G40/H^1/")

    driver.set_window_size(390, 844)
    time.sleep(1)
    return driver

In [22]:
def open_tab(driver,link):
    driver.execute_script(f"""window.open('{link}', "_blank");""")
    time.sleep(2)
    driver.switch_to.window(driver.window_handles[-1])

#Remove Loader
def remove_Loader(driver):
    driver.execute_script("document.querySelector('.bl-Preloader').remove();")

#Accept Cookies
def accept_cookies(driver):
    cookies = driver.find_elements(By.CSS_SELECTOR, ".ccm-CookieConsentPopup_Accept ")
    if(len(cookies) > 0):
        cookies[0].click()
    

#Sorry No Markets Error
def no_markets(driver):
    markets = driver.find_elements(By.CSS_SELECTOR, ".sm-NoAvailableMarkets_Header ")
    if(len(markets) > 0):
        open_tab(driver)

def sort_string(string):
    string = ''.join(e for e in string if e.isalnum())
    string = string.lower()
    return string

In [23]:
new_driver = driver_code()
open_tab(new_driver, 'https://www.bet365.com/#/AC/B1/C1/D1002/E91422157/G40/H^1/')
accept_cookies(new_driver)
time.sleep(1)
open_tab(new_driver, 'https://www.bet365.com/#/AC/B1/C1/D1002/E91422157/G40/H^1/')

teams = []
times = []
odds = []
dates = []
teams_ = new_driver.find_elements(
                By.CSS_SELECTOR, ".rcl-ParticipantFixtureDetailsTeam_TeamName "
                 )
times_ = new_driver.find_elements(
                By.CSS_SELECTOR, ".rcl-ParticipantFixtureDetails_BookCloses "
                )
for i in teams_:
    teams.append(i.text)
for i in times_:
    times.append(i.text)

odds_ = new_driver.find_elements(
    By.CSS_SELECTOR, ".sgl-ParticipantOddsOnly80_Odds"
    )

for key in odds_:
    odds.append(key.text)
    

home_teams = teams[::2]
away_teams = teams[1::2]
home_odds = odds[0:len(times)]
draw_odds = odds[len(times):len(times) * 2]
away_odds = odds[len(times)*2:len(odds)]

#Columns for Dataframe
columns = ['Home Team', 'Away Team',"Home Odds","Draw Odds","Away Odds"]

# Initialize a new DataFrame with columns
new_dataframe = pd.DataFrame(columns=columns)

# Add arrays to columns
new_dataframe['Home Team'] = home_teams
new_dataframe['Away Team'] = away_teams
new_dataframe['Home Odds'] = home_odds
new_dataframe['Draw Odds'] = draw_odds
new_dataframe['Away Odds'] = away_odds

new_driver.quit()

/var/folders/m2/30tqfzv54fx3fvq_xgkw697c0000gn/T/ipykernel_3988/1072383674.py:20: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(


In [24]:
new_dataframe

,Home Team,Away Team,Home Odds,Draw Odds,Away Odds
0,Everton,Aston Villa,17/10,5/2,31/20
1,Man Utd,Tottenham,11/10,14/5,21/10
2,Arsenal,Crystal Palace,3/10,9/2,9/1
3,Brentford,Nottm Forest,10/11,23/10,3/1
4,Sheff Utd,West Ham,5/2,9/4,11/10
5,Bournemouth,Liverpool,7/2,14/5,8/11
6,Brighton,Wolverhampton,8/13,14/5,17/4
7,Nottm Forest,Arsenal,13/2,3/1,9/20
8,Fulham,Everton,7/5,9/4,19/10
9,Luton,Brighton,15/4,10/3,8/13
